<span style="font-size:24px; font-family:'Roboto'; font-weight:bold;">
Script to visualize iMOD inputs in QGIS.
</span><br>

In [ ]:
print(f"\033[1m{'*'*50} Mdl_In_to_mm {'*'*50}\033[0m")
print('This script converts model inputs (mainly IDF) to TIF files, to be visualized in QGIS. It also calculates some parameters from Mdl Ins (e.g.) Aquifer layer thickness is calculated from TOP-BOT (same as MF does internally).')

## 0.0. Libraries

In [ ]:
from WS_Mdl import utils as U
from WS_Mdl import geo as G
from WS_Mdl import utils_imod as UIM

In [ ]:
import importlib
importlib.reload(G)
importlib.reload(U)
importlib.reload(UIM)

## 0.1. Options

In [ ]:
MdlN = 'NBr13'

# 3. Process Regular, Time Independent IDF Files

Select rows to be converted.

In [ ]:
G.PRJ_to_TIF(MdlN)

# 6. Time dependent packages
Those will have to be processed 1 by 1.

In [ ]:
d_paths = U.get_MdlN_paths(MdlN)

In [ ]:
for Pkg in DF.loc[ (DF["time"].notna()) & (DF['time']!='-'), 'package'].unique():
    print(DF.loc[ DF['package']==Pkg, 'path'].iloc[0])

In [ ]:
DF_time = DF[ ( DF["time"].notna() ) &
              ( DF["time"]!='-'    ) &
              ( DF['path'].notna() )] # Non time packages have NaN in 'time' Fld. Failed packages have '-', so they'll also be excluded. 

## 6.0. DRN & RIV

In [ ]:
import os
import re
import imod

In [ ]:

# -------------------- Get paths ------------------------------------------------------------------------------------------
d_paths = U.get_MdlN_paths(MdlN)
Xmin, Ymin, Xmax, Ymax, cellsize, N_R, N_C = U.Mdl_Dmns_from_INI(d_paths['path_INI'])

# -------------------- Read PRJ to DF -------------------------------------------------------------------------------------
DF = UIM.PRJ_to_DF(MdlN) # Read PRJ file to DF

In [ ]:
Mdl = 'NBr'

In [ ]:
print(f' --- Converting time dependant packages ---')
for i, R in DF_time[DF_time['package'].isin(('DRN', 'RIV'))].iterrows():
    print(f"\t{f"{R['package']}_{R['parameter']}":<30} ... ", end='')
    
    try:    
        ## Prepare directoreis and filenames
        path_TIF = os.path.join(d_paths['path_Mdl'], 'PoP', 'In', R['package'], R['MdlN'], os.path.basename(re.sub(r'\.idf$', '.tif', R['path'], flags=re.IGNORECASE)))  # Full path to TIF file
        os.makedirs(os.path.dirname(path_TIF), exist_ok=True) # Make sure the directory exists

        ## Build a dictionary mapping each band’s name to its row’s metadata.
        d_MtDt = {f"{R['parameter']}_L{R['layer']}_{R['MdlN']}" : {('origin_path' if col == 'path' else col): str(val) for col, val in R.items()}}

        DA = imod.formats.idf.open(R['path'], pattern=f'{{name}}_{Mdl}').sel(x=slice(Xmin, Xmax), y=slice(Ymax, Ymin))
        # Utl.DA_to_TIF(DA.squeeze(drop=True), path_TIF, d_MtDt) # .squeeze cause 2D arrays have extra dimension with size 1 sometimes.
        print(f'\u2713 - IDF converted to TIF - single-band without L attribute')
    except Exception as e:
        print(f"\u274C - Error: {e}")


In [ ]:
R['path']

## 6.1. WEL

In [ ]:
DF_WEL = DF.loc[DF['package']=='WEL']

In [ ]:
for i, R in DF_WEL.iloc[3:6].iterrows():
    print(f"\t{os.path.basename(R['path']):<30} ... ", end='\n')

    try:
        DF_IPF = imod.formats.ipf.read(R['path'])
        DF_IPF = DF_IPF.loc[ ( (DF_IPF['x']>Xmin) & (DF_IPF['x']<Xmax ) ) &
                            ( (DF_IPF['y']>Ymin) & (DF_IPF['y']<Ymax ) )].copy() # Slice to OBS within the Mdl Aa
        
        if ('q_m3' in DF_IPF.columns) and ('id' not in DF_IPF.columns):
            DF_IPF.rename(columns={'q_m3':'id'}, inplace=True) # One of the IPF files has q_m3 instead of id in it's fields. Don't ask me why, but it has to be dealt with.

        #666 I'll only save the average flow now
        DF_IPF_AVG = DF_IPF.groupby('id')[DF_IPF.select_dtypes(include=np.number).columns].agg(np.mean)
        _GDF_AVG = gpd.GeoDataFrame(DF_IPF_AVG, geometry=gpd.points_from_xy(DF_IPF_AVG['x'], DF_IPF_AVG['y'])).set_crs(crs=Utl.crs)

        path_GPKG = os.path.join(d_paths['path_Mdl'], 'PoP', 'In', R['package'], R['MdlN'], os.path.basename(re.sub(r'\.ipf$', '.gpkg', R['path'], flags=re.IGNORECASE)))  # Full path to TIF file    os.makedirs(os.path.dirname(path_GPKG), exist_ok=True) # Make sure the directory exists
        _GDF_AVG.to_file(path_GPKG, driver="GPKG") #, layer=os.path.basename(path_GPKG))
        print(f'\u2713 - IPF average values (per id) converted to GPKG')
    except:
        print('\u274C')

print(f' --- Success! ---')
print(f' {"-"*100}')

## 6.2. CHD --- TBC

# -1 Junkyard

### 2.0.1. extra (MetaSWAP files to be copied) Pkg needs to be read separetely cause of its unique format.

In [ ]:
if False: # Deactiated for now. Extra files are .inp, which is not a specific format. I'd have to process them individually, so I'm skipping this for now.
    DF.drop('EXTRA', inplace=True) # Drop the "failed to read package" row, as we're going to read it now.
    Pkg_name = 'extra'
    Pkg = d_PRJ[Pkg_name]

    l_paths = [str(i[0]) for i in Pkg['paths']]
    for path in l_paths:
        Par = path.split('/')[-3]

        Ln_DF_path = {"package": Pkg_name, "parameter": Par, 'path': path, 'active': True} 
        DF.loc[f'{Pkg_name.upper()}_{Par}'] = Ln_DF_path